# Recurrent neural networks

In the previous module, we covered rich semantic representations of text. The architecture we've been using captures the aggregated meaning of words in a sentence, but it does not take into account the **order** of the words, because the aggregation operation that follows the embeddings removes this information from the original text. Because these models are unable to represent word ordering, they cannot solve more complex or ambiguous tasks such as text generation or question answering.

To capture the meaning of a text sequence, we'll use a neural network architecture called **recurrent neural network**, or RNN. When using an RNN, we pass our sentence through the network one token at a time, and the network produces some **state**, which we then pass to the network again with the next token.

在上一个模块中，我们介绍了文本的丰富语义表示。 我们一直使用的架构捕获句子中单词的聚合含义，但它没有考虑单词的**顺序**，因为嵌入之后的聚合操作从原始文本中删除了此信息。 因为这些模型无法表示词序，所以它们无法解决更复杂或模棱两可的任务，例如文本生成或问答。

为了捕捉文本序列的含义，我们将使用一种称为**递归神经网络**或 RNN 的神经网络架构。 当使用 RNN 时，我们一次通过网络传递我们的句子一个标记，网络产生一些**状态**，然后我们用下一个标记再次传递给网络。

![Image showing an example recurrent neural network generation.](./images/rnn.png)

Given the input sequence of tokens $X_0,\dots,X_n$, the RNN creates a sequence of neural network blocks, and trains this sequence end-to-end using backpropagation. Each network block takes a pair $(X_i,S_i)$ as an input, and produces $S_{i+1}$ as a result. The final state $S_n$ or output $Y_n$ goes into a linear classifier to produce the result. All network blocks share the same weights, and are trained end-to-end using one backpropagation pass.

> The figure above shows recurrent neural network in the unrolled form (on the left), and in more compact recurrent representation (on the right). It is important to realize that all RNN Cells have the same **shareable weights**.

Because state vectors $S_0,\dots,S_n$ are passed through the network, the RNN is able to learn sequential dependencies between words. For example, when the word *not* appears somewhere in the sequence, it can learn to negate certain elements within the state vector.

Inside, each RNN cell contains two weight matrices: $W_H$ and $W_I$, and bias $b$. At each RNN step, given input $X_i$ and input state $S_i$, output state is calculated as $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, where $f$ is an activation function (often $\tanh$).

> For problems like text generation (that we will cover in the next unit) or machine translation we also want to get some output value at each RNN step. In this case, there is also another matrix $W_O$, and output is caluclated as $Y_i=f(W_O\times S_i+b_O)$.

Let's see how recurrent neural networks can help us classify our news dataset.

> For the sandbox environment, we need to run the following cell to make sure the required library is installed, and data is prefetched. If you are running locally, you can skip the following cell.


给定令牌 $X_0,\dots,X_n$ 的输入序列，RNN 创建一个神经网络块序列，并使用反向传播端到端地训练这个序列。 每个网络块将一对 $(X_i,S_i)$ 作为输入，并生成 $S_{i+1}$ 作为结果。 最终状态 $S_n$ 或输出 $Y_n$ 进入线性分类器以产生结果。 所有网络块共享相同的权重，并使用一个反向传播过程进行端到端训练。

> 上图显示了展开形式的循环神经网络（左侧）和更紧凑的循环表示形式（右侧）。 重要的是要认识到所有 RNN 单元都具有相同的**可共享权重**。

因为状态向量 $S_0,\dots,S_n$ 通过网络传递，RNN 能够学习单词之间的顺序依赖关系。 例如，当单词 *not* 出现在序列中的某处时，它可以学习否定状态向量中的某些元素。

在内部，每个 RNN 单元包含两个权重矩阵：$W_H$ 和 $W_I$，以及偏置 $b$。 在每个 RNN 步骤中，给定输入 $X_i$ 和输入状态 $S_i$，输出状态计算为 $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$，其中 $f$ 是一个激活函数（通常是 $\tanh$）。

> 对于文本生成（我们将在下一个单元中介绍）或机器翻译等问题，我们还希望在每个 RNN 步骤中获得一些输出值。 在这种情况下，还有另一个矩阵$W_O$，输出计算为$Y_i=f(W_O\times S_i+b_O)$。

让我们看看递归神经网络如何帮助我们对新闻数据集进行分类。

> 对于沙盒环境，我们需要运行以下单元以确保安装了所需的库，并预取了数据。 如果您在本地运行，则可以跳过以下单元格。

In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

When training large models, GPU memory allocation may become a problem. We also may need to experiment with different minibatch sizes, so that the data fits into our GPU memory, yet the training is fast enough. If you are running this code on your own GPU machine, you may experiment with adjusting minibatch size to speed up training.

> **Note**: Certain versions of NVidia drivers are known not to release the memory after training the model. We are running several examples in this notebooks, and it might cause memory to be exhausted in certain setups, especially if you are doing your own experiments as part of the same notebook. If you encounter some weird errors when starting to train the model, you may want to restart notebook kernel.

在训练大型模型时，GPU 内存分配可能会成为问题。 我们还可能需要尝试不同的小批量大小，以便数据适合我们的 GPU 内存，但训练速度足够快。 如果您在自己的 GPU 机器上运行此代码，您可以尝试调整小批量大小以加快训练速度。

> **注意**：已知某些版本的 NVidia 驱动程序在训练模型后不会释放内存。 我们在此笔记本中运行了几个示例，这可能会导致某些设置中的内存耗尽，尤其是当您将自己的实验作为同一笔记本的一部分进行时。 如果您在开始训练模型时遇到一些奇怪的错误，您可能需要重启笔记本内核。

In [3]:
batch_size = 16
embed_size = 64

## Simple RNN classifier

In the case of a simple RNN, each recurrent unit is a simple linear network, which takes in an input vector and state vector, and produces a new state vector. In Keras, this can be represented by the `SimpleRNN` layer.

While we can pass one-hot encoded tokens to the RNN layer directly, this is not a good idea because of their high dimensionality. Therefore, we will use an embedding layer to lower the dimensionality of word vectors, followed by an RNN layer, and finally a `Dense` classifier.

> **Note**: In cases where the dimensionality isn't so high, for example when using character-level tokenization, it might make sense to pass one-hot encoded tokens directly into the RNN cell.


在简单 RNN 的情况下，每个循环单元都是一个简单的线性网络，它接收输入向量和状态向量，并产生一个新的状态向量。 在 Keras 中，这可以由 `SimpleRNN` 层表示。

虽然我们可以将 one-hot 编码的令牌直接传递给 RNN 层，但这不是一个好主意，因为它们的维度很高。 因此，我们将使用一个嵌入层来降低词向量的维数，然后是一个 RNN 层，最后是一个`Dense`分类器。

> **注意**：在维数不是那么高的情况下，例如当使用字符级标记化时，将单热编码标记直接传递到 RNN 单元可能是有意义的。

In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Note:** We use an untrained embedding layer here for simplicity, but for better results we can use a pretrained embedding layer using Word2Vec, as described in the previous unit. It would be a good exercise for you to adapt this code to work with pretrained embeddings.

Now let's train our RNN. RNNs in general are quite difficult to train, because once the RNN cells are unrolled along the sequence length, the resulting number of layers involved in backpropagation is quite large. Thus we need to select a smaller learning rate, and train the network on a larger dataset to produce good results. This can take quite a long time, so using a GPU is preferred.

To speed things up, we will only train the RNN model on news titles, omitting the description. You can try training with description and see if you can get the model to train.

> **注意：** 为简单起见，我们在这里使用未经训练的嵌入层，但为了获得更好的结果，我们可以使用使用 Word2Vec 的预训练嵌入层，如上一单元所述。 调整此代码以使用预训练嵌入对您来说将是一个很好的练习。

现在让我们训练我们的 RNN。 RNN 通常很难训练，因为一旦 RNN 单元沿序列长度展开，反向传播所涉及的层数就会非常大。 因此我们需要选择较小的学习率，并在较大的数据集上训练网络以产生良好的结果。 这可能需要相当长的时间，因此最好使用 GPU。

为了加快速度，我们将只在新闻标题上训练 RNN 模型，省略描述。 您可以尝试使用描述进行训练，看看是否可以让模型进行训练。

In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Note** that accuracy is likely to be lower here, because we are training only on news titles.

zh

> **注意** 这里的准确度可能较低，因为我们只针对新闻标题进行训练。

## Revisiting variable sequences 

Remember that the `TextVectorization` layer will automatically pad sequences of variable length in a minibatch with pad tokens. It turns out that those tokens also take part in training, and they can complicate convergence of the model.

There are several approaches we can take to minimize the amount of padding. One of them is to reorder the dataset by sequence length and group all sequences by size. This can be done using the `tf.data.experimental.bucket_by_sequence_length` function (see [documentation](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)). 

Another approach is to use **masking**. In Keras, some layers support additional input that shows which tokens should be taken into account when training. To incorporate masking into our model, we can either include a separate `Masking` layer ([docs](https://keras.io/api/layers/core_layers/masking/)), or we can specify the `mask_zero=True` parameter of our `Embedding` layer.

> **Note**: This training will take around 5 minutes to complete one epoch on the whole dataset. Feel free to interrupt training at any time if you run out of patience. What you can also do is limit the amount of data used for training, by adding `.take(...)` clause after `ds_train` and `ds_test` datasets.

请记住，`TextVectorization` 层将自动使用填充令牌填充小批量中的可变长度序列。 事实证明，这些标记也参与了训练，它们会使模型的收敛变得复杂。

我们可以采用多种方法来最小化填充量。 其中之一是按序列长度对数据集重新排序，并按大小对所有序列进行分组。 这可以使用`tf.data.experimental.bucket_by_sequence_length`函数来完成（参见[文档](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)）。

另一种方法是使用**掩码**。 在 Keras 中，一些层支持额外的输入，显示训练时应考虑哪些标记。 要将掩蔽合并到我们的模型中，我们可以包括一个单独的“掩蔽”层 ([docs](https://keras.io/api/layers/core_layers/masking/))，或者我们可以指定`mask_zero=True` 我们的 `Embedding` 层的 ` 参数。

> **注意**：此训练大约需要 5 分钟才能在整个数据集上完成一个 epoch。 如果您没有耐心，请随时中断训练。 您还可以通过在 `ds_train` 和 `ds_test` 数据集之后添加 `.take(...) `子句来限制用于训练的数据量。

In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Now that we're using masking, we can train the model on the whole dataset of titles and descriptions.

> **Note**: Have you noticed that we have been using vectorizer trained on the news titles, and not the whole body of the article? Potentially, this can cause some of the the tokens to be ignored, so it is better to re-train the vectorizer. However, it might only have very small effect, so we will stick to the previous pre-trained vectorizer for the sake of simplicity.

现在我们正在使用掩码，我们可以在标题和描述的整个数据集上训练模型。

> **注意**：您是否注意到我们一直在使用针对新闻标题而非整篇文章训练的向量化器？ 这可能会导致某些标记被忽略，因此最好重新训练向量化器。 然而，它可能只有很小的影响，所以为了简单起见，我们将坚持使用之前的预训练向量化器。

## LSTM: Long short-term memory

One of the main problems of RNNs is **vanishing gradients**. RNNs can be pretty long, and may have a hard time propagating the gradients all the way back to the first layer of the network during backpropagation. When this happens, the network cannot learn relationships between distant tokens. One way to avoid this problem is to introduce **explicit state management** by using **gates**. The two most common architectures that introduce gates are **long short-term memory** (LSTM) and **gated relay unit** (GRU). We'll cover LSTMs here.

RNN 的主要问题之一是**梯度消失**。 RNN 可能很长，并且在反向传播过程中可能很难将梯度一直传播回网络的第一层。 发生这种情况时，网络无法学习远距离令牌之间的关系。 避免此问题的一种方法是通过使用 **gates** 引入 **显式状态管理**。 引入门的两种最常见的架构是**长短期记忆**（LSTM）和**门控中继单元**（GRU）。 我们将在这里介绍 LSTM。

![Image showing an example long short term memory cell](./images/long-short-term-memory-cell.svg)

An LSTM network is organized in a manner similar to an RNN, but there are two states that are passed from layer to layer: the actual state $c$, and the hidden vector $h$. At each unit, the hidden vector $h_{t-1}$ is combined with input $x_t$, and together they control what happens to the state $c_t$ and output $h_{t}$ through **gates**. Each gate has sigmoid activation (output in the range $[0,1]$), which can be thought of as a bitwise mask when multiplied by the state vector. LSTMs have the following gates (from left to right on the picture above):
* **forget gate** which determines which components of the vector $c_{t-1}$ we need to forget, and which to pass through. 
* **input gate** which determines how much information from the input vector and previous hidden vector should be incorporated into the state vector.
* **output gate** which takes the new state vector and decides which of its components will be used to produce the new hidden vector $h_t$.

LSTM 网络的组织方式类似于 RNN，但有两种状态在层与层之间传递：实际状态 $c$ 和隐藏向量 $h$。 在每个单元，隐藏向量 $h_{t-1}$ 与输入 $x_t$ 组合，它们一起控制状态 $c_t$ 发生的事情，并通过 **门** 输出 $h_{t}$。 每个门都有 sigmoid 激活（输出在 $[0,1]$ 范围内），当与状态向量相乘时，可以将其视为按位掩码。 LSTM 具有以下门（上图从左到右）：
* **遗忘门**，它决定了向量 $c_{t-1}$ 的哪些分量我们需要遗忘，哪些要通过。
* **输入门**，它决定了应将来自输入向量和先前隐藏向量的多少信息合并到状态向量中。
* **输出门** 它采用新的状态向量并决定其哪些组件将用于生成新的隐藏向量 $h_t$。

The components of the state $c$ can be thought of as flags that can be switched on and off. For example, when we encounter the name *Alice* in the sequence, we guess that it refers to a woman, and raise the flag in the state that says we have a female noun in the sentence. When we further encounter the words *and Tom*, we will raise the flag that says we have a plural noun. Thus by manipulating state we can keep track of the grammatical properties of the sentence.

> **Note**: Here's a great resource for understanding the internals of LSTMs: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) by Christopher Olah.

While the internal structure of an LSTM cell may look complex, Keras hides this implementation inside the `LSTM` layer, so the only thing we need to do in the example above is to replace the recurrent layer:

状态 $c$ 的组件可以被认为是可以打开和关闭的标志。 例如，当我们在序列中遇到名字 *Alice* 时，我们猜测它指的是一个女人，并在表示我们在句子中有一个女性名词的状态下举起旗帜。 当我们进一步遇到单词*和 Tom* 时，我们将举起旗帜，表明我们有一个复数名词。 因此，通过操纵状态，我们可以跟踪句子的语法属性。

> **注意**：这是了解 LSTM 内部结构的重要资源：Christopher Olah 的[了解 LSTM 网络](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)。

虽然 LSTM 单元的内部结构可能看起来很复杂，但 Keras 将此实现隐藏在 `LSTM` 层中，因此我们在上面的示例中唯一需要做的就是替换循环层：

In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


> **Note** that training LSTMs is also quite slow, and you may not seem much increase in accuracy in the beginning of training. You may need to continue training for some time to achieve good accuracy.

zh

> **注意** 训练 LSTMs 也很慢，在训练开始时你的准确率似乎没有太大提高。 您可能需要继续训练一段时间才能达到良好的准确性。

## Bidirectional and multilayer RNNs

In our examples so far, the recurrent networks operate from the beginning of a sequence until the end. This feels natural to us because it follows the same direction in which we read or listen to speech. However, for scenarios which require random access of the input sequence, it makes more sense to run the recurrent computation in both directions. RNNs that allow computations in both directions are called **bidirectional** RNNs, and they can be created by wrapping the recurrent layer with a special `Bidirectonal` layer.

> **Note**: The `Bidirectional` layer makes two copies of the layer within it, and sets the `go_backwards` property of one of those copies to `True`, making it go in the opposite direction along the sequence.

Recurrent networks, unidirectional or bidirectional, capture patterns within a sequence, and store them into state vectors or return them as output. As with convolutional networks, we can build another recurrent layer following the first one to capture higher level patterns, built from lower level patterns extracted by the first layer. This leads us to the notion of a **multi-layer RNN**, which consists of two or more recurrent networks, where the output of the previous layer is passed to the next layer as input.

到目前为止，在我们的示例中，循环网络从序列的开头一直运行到结尾。 这对我们来说感觉很自然，因为它遵循我们阅读或听演讲的相同方向。 然而，对于需要随机访问输入序列的场景，在两个方向上运行循环计算更有意义。 允许双向计算的 RNN 称为 **双向** RNN，它们可以通过用特殊的`Bidirectional`层包裹循环层来创建。

> **注意**：`Bidirectional` 层在其中复制了两个图层，并将其中一个副本的 `go_backwards` 属性设置为 `True`，使其沿着序列向相反的方向移动。

单向或双向循环网络捕获序列中的模式，并将它们存储到状态向量中或将它们作为输出返回。 与卷积网络一样，我们可以在第一个循环层之后构建另一个循环层以捕获更高级别的模式，该模式是从第一层提取的较低级别模式构建的。 这使我们想到了**多层 RNN** 的概念，它由两个或多个循环网络组成，其中前一层的输出作为输入传递到下一层。

![Image showing a Multilayer long-short-term-memory- RNN](./images/multi-layer-lstm.jpg)

*Picture from [this wonderful post](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) by Fernando López.*

Keras makes constructing these networks an easy task, because you just need to add more recurrent layers to the model. For all layers except the last one, we need to specify `return_sequences=True` parameter, because we need the layer to return all intermediate states, and not just the final state of the recurrent computation.

Let's build a two-layer bidirectional LSTM for our classification problem.

> **Note** this code again takes quite a long time to complete, but it gives us highest accuracy we have seen so far. So maybe it is worth waiting and seeing the result.

*图片来自 Fernando López 的[这篇精彩文章](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3)。*

Keras 使构建这些网络成为一项简单的任务，因为您只需要向模型添加更多循环层。 对于除最后一层之外的所有层，我们需要指定 `return_sequences=True` 参数，因为我们需要该层返回所有中间状态，而不仅仅是循环计算的最终状态。

让我们为我们的分类问题构建一个双层双向 LSTM。

> **注意** 此代码再次需要相当长的时间才能完成，但它为我们提供了迄今为止我们所见的最高准确度。 因此，也许值得等待并查看结果。

In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNNs for other tasks

Up until now, we've focused on using RNNs to classify sequences of text. But they can handle many more tasks, such as text generation and machine translation &mdash; we'll consider those tasks in the next unit.

到目前为止，我们一直专注于使用 RNN 对文本序列进行分类。 但它们可以处理更多任务，例如文本生成和机器翻译—— 我们将在下一个单元中考虑这些任务。